In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
import h5py
import tensorflow as tf
# from google.colab import drive
# drive.mount('/content/drive/')
# data_dir = 'C:\Users\claus\Downloads'
data_dir ='biological_data_pfpp/biological_data_pfp/test'


C:\Users\claud\AppData\Local\Temp\ipykernel_11652\1329629862.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
from tensorflow import keras
from tensorflow.keras.models import load_model

In [7]:
model_bp=load_model('model_bp.keras')
model_mf=load_model('model_mf.keras')
model_cc =load_model('model_cc.keras')

ValueError: Layer 'batch_normalization_9' expected 4 variables, but received 0 variables during loading. Expected: ['batch_normalization_9/gamma:0', 'batch_normalization_9/beta:0', 'batch_normalization_9/moving_mean:0', 'batch_normalization_9/moving_variance:0']

In [7]:
test_embeddings = []
test_protein_ids = []

with h5py.File(data_dir+'/test_embeddings.h5', 'r') as f:
    for protein_id in f.keys():  # protein ids
        embeddings = f[protein_id][:]
        test_embeddings.append(embeddings)
        test_protein_ids.append(protein_id)

test_embeddings_array = np.array(test_embeddings)

In [16]:
test_embeddings_array.shape

(1000, 1024)

In [18]:
column_num = test_embeddings_array.shape[1]
test_df = pd.DataFrame(test_embeddings_array, columns=["Column_" + str(i) for i in range(1, column_num+1)])

# Concatenating protein_ids with the embeddings DataFrame
test_df = pd.concat((pd.DataFrame(test_protein_ids, columns=['Protein_ID']), test_df), axis=1)
#print(test_df.head())

print(test_df.shape)
print(test_df)

(1000, 1025)
     Protein_ID  Column_1  Column_2  Column_3  Column_4  Column_5  Column_6  \
0    A0A0B4JCV4  0.009789 -0.039734  0.036530 -0.006447 -0.040619  0.040436   
1    A0A0B4KHT0  0.027863 -0.011543  0.008865 -0.017654  0.007339  0.031769   
2    A0A0B4P506  0.016434  0.018021  0.037018 -0.059113  0.035614  0.008308   
3    A0A0G2K1A2  0.008820  0.083496 -0.001374 -0.000365 -0.064209  0.008026   
4    A0A0G2K1V4  0.065918  0.092896 -0.001803  0.022598  0.038300  0.049957   
..          ...       ...       ...       ...       ...       ...       ...   
995      Q9ZP06  0.019211  0.082825  0.026566  0.021088  0.014359  0.046783   
996      Q9ZVF3  0.038940 -0.071594  0.016251 -0.012314  0.011696  0.104248   
997      S0HPF7  0.064331  0.066895  0.024521  0.027161  0.023041  0.059448   
998      W5U9R6  0.033508  0.015961  0.026474  0.027298 -0.016602  0.025864   
999      W7K139  0.028732  0.037140 -0.004421  0.019913  0.025726  0.082397   

     Column_7  Column_8  Column_9  ...

In [39]:
predictions_cc = model_cc.predict(test_df.iloc[:, 1:])
predictions_mf = model_mf.predict(test_df.iloc[:, 1:])
predictions_bp = model_bp.predict(test_df.iloc[:, 1:])
# print(predictions)
# predictions.shape

32/32 [==============================] - 0s 4ms/step


In [26]:
type(predictions_cc)

numpy.ndarray

In [40]:
total_prediction=np.concatenate((predictions_cc,predictions_bp,predictions_mf), axis=1)
print(total_prediction.shape)

(1000, 3004)


In [32]:
train_set = pd.read_csv('biological_data_pfpp/biological_data_pfp/train'+'/train_set.tsv', delimiter='\t')
print(train_set.shape)
train_set.head()

(4277047, 3)


,Protein_ID,aspect,GO_term
0,P91124,cellular_component,GO:0005575
1,P91124,cellular_component,GO:0110165
2,P91124,cellular_component,GO:0005737
3,P91124,cellular_component,GO:0005622
4,P91124,cellular_component,GO:0043226


In [41]:
train_set_cc=train_set[train_set['aspect']=='cellular_component']
labels_cc = train_set_cc['GO_term'].value_counts().index[:].tolist()
# labels_cc
train_set_mf=train_set[train_set['aspect']=='molecular_function']
labels_mf = train_set_mf['GO_term'].value_counts().index[:].tolist()
# labels_cc
train_set_bp=train_set[train_set['aspect']=='biological_process']
labels_bp = train_set_bp['GO_term'].value_counts().index[:].tolist()
# labels_cc
labels=labels_cc+labels_mf+labels_bp
print(len(labels))

3004


In [42]:
pred_df=pd.DataFrame(total_prediction,columns=labels)

In [62]:
pred_df_p = pd.concat((pd.DataFrame(test_protein_ids, columns=['Protein_ID']), pred_df), axis=1)
pred_df_p

,Protein_ID,GO:0005575,GO:0110165,GO:0005622,GO:0043226,GO:0043229,GO:0043227,GO:0005737,GO:0043231,GO:0005634,...,GO:0008356,GO:0032680,GO:0030522,GO:0048546,GO:0002702,GO:0072073,GO:1901989,GO:0061014,GO:0006814,GO:0019722
0,A0A0B4JCV4,0.500161,0.501672,0.494457,0.502443,0.499571,0.500447,0.501586,0.496628,0.500103,...,0.499393,0.502447,0.505622,0.496511,0.506677,0.502197,0.506408,0.500033,0.497170,0.496140
1,A0A0B4KHT0,0.497802,0.501790,0.493724,0.500765,0.499637,0.504167,0.502865,0.501539,0.501424,...,0.497811,0.501747,0.503738,0.496442,0.505952,0.498987,0.505854,0.498774,0.498865,0.493411
2,A0A0B4P506,0.501235,0.497948,0.495091,0.500550,0.500561,0.498864,0.498190,0.501936,0.498927,...,0.497323,0.504739,0.502207,0.498678,0.505284,0.498610,0.502679,0.497111,0.499729,0.498984
3,A0A0G2K1A2,0.500019,0.498565,0.493031,0.500547,0.500416,0.505123,0.502643,0.502805,0.501734,...,0.498086,0.506826,0.501335,0.498788,0.509511,0.502831,0.503365,0.498278,0.497817,0.493789
4,A0A0G2K1V4,0.501641,0.504668,0.495047,0.502296,0.497247,0.504750,0.503658,0.499995,0.499472,...,0.499288,0.505590,0.500427,0.498230,0.509669,0.502751,0.501602,0.500103,0.503755,0.494331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Q9ZP06,0.497756,0.503036,0.495604,0.502046,0.498978,0.498922,0.504568,0.501827,0.501634,...,0.499091,0.504320,0.498562,0.498476,0.507902,0.501827,0.497176,0.500913,0.499921,0.495947
996,Q9ZVF3,0.500519,0.498830,0.494263,0.509322,0.502349,0.500871,0.504516,0.500747,0.496548,...,0.500390,0.501867,0.500461,0.497281,0.508283,0.499296,0.504929,0.500883,0.499845,0.494574
997,S0HPF7,0.498955,0.500033,0.492947,0.497811,0.499427,0.502642,0.499422,0.500354,0.497332,...,0.500069,0.505307,0.502814,0.497542,0.509056,0.501682,0.501917,0.498753,0.499463,0.493601
998,W5U9R6,0.501493,0.501080,0.493055,0.501329,0.498602,0.498883,0.498624,0.503009,0.500348,...,0.498354,0.502433,0.500549,0.498418,0.505373,0.500244,0.500974,0.500924,0.499366,0.495430


In [56]:
dfghj=[['ess',34,3],['sdfs',45,45]]

sdfds=pd.DataFrame(dfghj)
print(sdfds)

      0   1   2
0   ess  34   3
1  sdfs  45  45


In [64]:
pred_df_p['GO:0030117']

0      0.487975
1      0.494836
2      0.499522
3      0.493572
4      0.487196
         ...   
995    0.496120
996    0.490930
997    0.494731
998    0.496846
999    0.492329
Name: GO:0030117, Length: 1000, dtype: float32

In [74]:
result=[]
for i in range(1000):
    if i%2==0:
        print(i)
    line=[pred_df_p.iloc[i][0]]
    # for j in range(3004):
        # print(pred_df_p.iloc[i][j+1])
    top_1500_columns = pred_df_p.iloc[i][1:3004].sort_values(ascending=False).head(1500).index
    for j in range(1500):
        line.append(top_1500_columns[j])
        line.append(pred_df_p.iloc[i][top_1500_columns[j]])
        # print(line)
        result.append(line)
# Select these columns
    # result_df = df[top_500_columns]

    # result_df #now contains the 500 columns with the highest values for the specified row
    # print(best)
    # fdfweewfwef
print(result)
print(pd.DataFrame(result))


0


C:\Users\claud\AppData\Local\Temp\ipykernel_11652\1651585290.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  line=[pred_df_p.iloc[i][0]]


2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446


In [ ]:
pd.DataFrame(result).to_csv('submission.tsv',header=None, index=False,sep='\t')

In [ ]:
import cafaeval
from cafaeval.evaluation import cafa_eval, write_results
res = cafa_eval(pred_dir = "" , gt_file="ground_truth.tsv")
write_results(*res)